<style TYPE="text/css">
code.has-jax {font: inherit; font-size: 100%; background: inherit; border: inherit;}
</style>
<script type="text/x-mathjax-config">
MathJax.Hub.Config({
    tex2jax: {
        inlineMath: [['$','$'], ['\\(','\\)']],
        skipTags: ['script', 'noscript', 'style', 'textarea', 'pre'] // removed 'code' entry
    }
});
MathJax.Hub.Queue(function() {
    var all = MathJax.Hub.getAllJax(), i;
    for(i = 0; i < all.length; i += 1) {
        all[i].SourceElement().parentNode.className += ' has-jax';
    }
});
</script>
<script type="text/javascript" src="http://cdn.mathjax.org/mathjax/latest/MathJax.js?config=TeX-AMS-MML_HTMLorMML"></script>



## Application of Reduced Gradient - Topology Optimization

<img src="tutorial_topopt/infill.gif" alt="Drawing" style="height: 600px;"/>

### Introduction

In this project, you will learn to implement an optimization algorithm for 
minimizing the compliance of a structure at its equilibrium state with respect to
its topology. The tutorial is based on Dr. Sigmund's topology optimization 
[code](http://www.topopt.dtu.dk/?q=node/751) 
and [paper](http://www.topopt.dtu.dk/files/TopOpt88.pdf). The template code is attached at the end of the tutorial.


### The compliance minimization problem
Topology optimization has been commonly used to design structures and materials with optimal mechanical, 
thermal, electromagnetic, acoustical, or other properties. 
The structure under design is segmented into $n$ finite elements, and a 
density value $x_i$ is assigned to each element $i \in \{1,2,...,n\}$: A higher density corresponds to a less porous material 
element and higher Yong's modulus. Reducing the density to zero is equivalent to creating a hole in the 
structure. Thus, the set of densities ${\bf x}=\{x_i\}$ can be used to represent the topology of the 
structure and is considered as the variables to be optimized. A common topology optimization problem is 
compliance minimization, where we seek the "stiffest" structure within a certain volume limit to withhold 
a particular load: 

$$ \min_{\bf x} \quad {\bf f} := {\bf d}^T {\bf K}({\bf x}) {\bf d} $$

$$ \text{subject to:} \quad {\bf h} := {\bf K}({\bf x}) {\bf d} = {\bf u}, $$

$$ \quad {\bf g} := V(\textbf{x}) \leq v,$$

$$ \textbf{x} \in [0,1]. $$

Here $V(\textbf{x})$ is the total volume; $v$ is an upper bound on volume; 
${\bf d} \in \mathbb{R}^{n_d\times 1}$ is the displacement of the structure under the load ${\bf u}$, 
where $n_d$ is the degrees of freedom (DOF) of the system (i.e., the number of x- and y-coordinates 
of nodes from the finite element model of the structure);
${\bf K(x)}$ is the global stiffness matrix for the structure.

${\bf K(x)}$ is indirectly influenced by the topology ${\bf x}$, through the element-wise stiffness matrix 

$${\bf K}_i = \bar{\bf K}_e E(x_i),$$

and the local-to-global assembly: 

$${\bf K(x)} = {\bf G}[{\bf K}_1,{\bf K}_2,...,{\bf K}_n],$$

where the matrix $\bar{\bf K}_e$ is predefined according to the finite element type (we use first-order 
quadrilateral elements throughout this tutorial) and the nodal displacements of the element (we use 
square elements with unit lengths throughout this tutorial), ${\bf G}$ is an assembly matrix, $E(x_i)$ is 
the element-wise Young's modulus defined as a function of the density $x_i$: 
$E(x_i) := \Delta E x_i^p + E_{\text{min}}$, where $p$ (the penalty parameter) is usually set to 3.
 This cubic relationship between the topology and the modulus 
is determined by the material constitutive models, and numerically, it 
also helps binarize the topologies, i.e., to push the optimal ${\bf x}_i$ to 1 or 0 (why?). 
The term $E_{\text{min}}$ is added to provide numerical stability.

### Design sensitivity analysis
This problem has both inequality and equality constraints. However, the inequality ones are only related
to the topology $\textbf{x}$, and are either 
linear ($V(\textbf{x}) \leq v$) or simple bounds ($\textbf{x} \in [0,1]$). We will 
show that these constraints can be easily handled.
The problem thus involves two sets of variables: We can consider $\textbf{x}$ as the **decision variables** 
and $\textbf{u}$ as the state variables that are governed by $\textbf{x}$ through the equality constraint
${\bf K}({\bf x}) {\bf d} = {\bf u}$.

The reduced gradient (often called design sensitivity) can be calculated as

$$\frac{df}{d{\bf x}} = \frac{\partial f}{\partial {\bf x}} - 
\frac{\partial f}{\partial {\bf u}}(\frac{\partial {\bf h}}{\partial {\bf u}})^{-1}
\frac{\partial {\bf h}}{\partial {\bf x}},$$

which leads to 

$$\frac{df}{d{\bf x}} = -{\bf u}^T \frac{\partial {\bf K}}{\partial {\bf x}}{\bf u}.$$

Recall the relation between ${\bf K}$ and ${\bf x}$, and notice that 

$${\bf u}^T {\bf K}{\bf u} = \sum_{i=1}^n {\bf u}^T_i{\bf K}_i{\bf u}_i,$$ 

i.e., the total compliance (strain energy) is the summation of element-wise compliance. 
We can further simplify the gradient as follows:

$$-{\bf u}^T \frac{\partial {\bf K}}{\partial {\bf x}}{\bf u} = - \frac{\partial {\bf u}^T{\bf K}{\bf u}}{\partial {\bf x}}$$

$$ = - \frac{\partial \sum_{i=1}^n {\bf u}^T_i{\bf K}_i{\bf u}_i}{\partial {\bf x}}$$

$$ = [..., - \frac{\partial {\bf u}^T_i{\bf K}_i{\bf u}_i}{\partial x_i}, ...] $$

$$ = [..., - {\bf u}^T_i \frac{\partial {\bf K}_i}{\partial x_i}{\bf u}_i, ...]$$

$$ = [..., - {\bf u}^T_i \frac{\partial \bar{\bf K}_e \Delta E x_i^3}{\partial x_i}{\bf u}_i, ...]$$

$$ = [..., - 3\Delta E x_i^2 {\bf u}^T_i \bar{\bf K}_e {\bf u}_i, ...]$$

### The algorithm
The pseudo code for compliance minimization is the following:


1. Problem setup (see details below)

2. Algorithm setup: $\epsilon = 0.001$ (or any small positive number), $k = 0$ (counter for the iteration),
$\Delta x = 1000$ (or any number larger than $\epsilon$)

3. While $norm(\Delta x) \leq \epsilon$, Do:
    
    3.1. Update the stiffness matrix ${\bf K}$ and the displacement (state) ${\bf u}$ (finite element analysis)
    
    3.2. Calculate element-wise compliance ${\bf u}^T_i \bar{\bf K}_e {\bf u}_i$
    
    3.3. Calculate partial derivatives 
    $$\frac{df}{dx_i} = - 3\Delta E x_i^2 {\bf u}^T_i \bar{\bf K}_e {\bf u}_i$$
    
    3.4. The gradient with respect to $g$ is a constant vector $[1,...,1]^T$
    
    3.5. Apply filter to $\frac{df}{d{\bf x}}$ (See discussion later)
    
    3.6. Update ${\bf x}$: $${\bf x}'_{k+1} = {\bf x}_{k} - \alpha_k (\frac{df}{d{\bf x}} + \mu {\bf 1})$$,
    where $\mu \geq 0$ is determined as in 3.7. To ensure that the gradient descent is successful, we will either set
    $\alpha_k$ to a small value, or truncate $\Delta x = - (\frac{df}{d{\bf x}} + \mu {\bf 1})$ within a range 
    (conceptually similar to the idea of trust region).
         
    3.7. Move ${\bf x}'_{k+1}$ back to the feasible domain:
    If ${\bf 1}^T{\bf x}_{k} < v$ and $-{\bf 1}^T\frac{df}{d{\bf x}}<0$, then ${\bf x}'_{k+1}$ satisfies $g$.
     with $\mu = 0$. If ${\bf x}'_{k+1}$ does not satisfy $g$, we will increase $\mu$ using bisection, i.e.,
     search in $[0,\mu_{max}]$ where $\mu_{max}$ is a large positive number. Also, we 
     will truncate ${\bf x}'_{k+1}$ between 0 and 1. 

    3.8. Update $norm(\Delta x)$, $k = k+1$

### Implementation details
Some details of the template code is explained as follows:

#### The numbering rule for elements and nodes
The template code assumes a rectangular design domain, where each element is modeled as a unit square. 
The numbering of elements and nodes are explained in the figure below.

<img src="tutorial_topopt/topopt881.png" alt="Drawing" style="height: 300px;"/> 


#### Input parameters
Inputs to the program are (1) the number of elements in the x and y directions (```nelx``` and ```nely```), 
(2) the volume fraction (```volfrac```, this is the number between 0 and 1 that specifies the
ratio between the volume of the target topology and the maximum volume $nelx \times nely$), (3) the penalty
parameter of the Young's Modulus model (```penal```, usually **=3**), and (4) the filter radius (```rmin```, 
usually **=3**).

#### Material properties
{% highlight MATLAB %}
%% MATERIAL PROPERTIES
E0 = 1;
Emin = 1e-9;
nu = 0.3;
{% endhighlight %}

Set the Young's Modulus (```E0```), and the Poisson's ratio (```nu```). Leave Emin as a small number.
  
#### Define loadings
{% highlight MATLAB %}
F = sparse(2,1,-1,2*(nely+1)*(nelx+1),1);
{% endhighlight %}

This line specifies the loading. Here ```F``` is a sparse column vector with ```2(nely+1)(nelx+1)``` elements.
```(2,1,-1,\cdots)``` specifies that there is a force of **-1** at the second row and first column
of the vector. According to the numbering convention of this code, this is to say that in the y direction
of the first node, there is a downward force of magnitude 1.

#### Define boundary conditions (Fixed DOFs)
{% highlight MATLAB %}
fixeddofs = union([1:2:2*(nely+1)],[2*(nelx+1)*(nely+1)]);
{% endhighlight %}

This line specifies the nodes with fixed DOFs. ```[1:2:2*(nely+1)]``` are x directions 
of all nodes to the left side of the structure, and ```2*(nelx+1)*(nely+1)``` is the y direction
of the last node (right bottom corner). See figure below:

<img src="tutorial_topopt/topopt882.png" alt="Drawing" style="height: 300px;"/> 

#### Filtering of sensitivity (gradient)
Pure gradient descent may result in a topology with checkerboard patterns.
While mathematically sound, such a solution can be infeasible from a manufacturing perspective or 
too expensive to realize (e.g., through additive manufacturing of porous structures). Therefore, 
a smoothed solution is often more preferred. 

---
### Topology Optimization Solution

We give the defined boundary condition to see the topology optimization: set fix DOFs at the left side of the beam and keep rest parts free. Then implement loading for the center and right side of the beam (see below figure). The input variables for the topology optimization are as below:

- Number of element in x: 150
- Number of element in y: 50
- Volume Fraction: 0.6
- Penalty: 3.0
- Filter-radius: 2
- Filter Method: Sensitivity filtering, which is set as 1

Below is the MATLAB code with objective function and boundary condition setting. 

![Image](https://github.com/dayanbatuofu/MAE-598-Design-Optimization/raw/main/design%20domain.png)

Defined loading is `F = sparse([2*(nely+1)*nelx+2,2*(nely+1)*(nelx/2)+2*(nely+1)],[1 2],[1 -1],2*(nely+1)*(nelx+1),2)`

Defined displacement is `zeros(2*(nely+1)*(nelx+1),2)`

Defined fixed DOFs is `fixeddofs = [1:2*nely+1]`

In [2]:
%%file top88.m
%plot inline

%%%% AN 88 LINE TOPOLOGY OPTIMIZATION CODE Nov, 2010 %%%%
function top88(nelx,nely,volfrac,penal,rmin,ft)
%% MATERIAL PROPERTIES
E0 = 1;
Emin = 1e-9;
nu = 0.3;
%% PREPARE FINITE ELEMENT ANALYSIS
A11 = [12  3 -6 -3;  3 12  3  0; -6  3 12 -3; -3  0 -3 12];
A12 = [-6 -3  0  3; -3 -6 -3 -6;  0 -3 -6  3;  3 -6  3 -6];
B11 = [-4  3 -2  9;  3 -4 -9  4; -2 -9 -4 -3;  9  4 -3 -4];
B12 = [ 2 -3  4 -9; -3  2  9 -2;  4  9  2  3; -9 -2  3  2];
KE = 1/(1-nu^2)/24*([A11 A12;A12' A11]+nu*[B11 B12;B12' B11]);
nodenrs = reshape(1:(1+nelx)*(1+nely),1+nely,1+nelx);
edofVec = reshape(2*nodenrs(1:end-1,1:end-1)+1,nelx*nely,1);
edofMat = repmat(edofVec,1,8)+repmat([0 1 2*nely+[2 3 0 1] -2 -1],nelx*nely,1);
iK = reshape(kron(edofMat,ones(8,1))',64*nelx*nely,1);
jK = reshape(kron(edofMat,ones(1,8))',64*nelx*nely,1);
% DEFINE LOADS AND SUPPORTS (HALF MBB-BEAM)
F = sparse([2*(nely+1)*nelx+2,2*(nely+1)*nelx+2*nely],[1 2],[1 -1],2*(nely+1)*(nelx+1),2);
U = zeros(2*(nely+1)*(nelx+1),2);
fixeddofs = [1:2*nely+1];
alldofs = [1:2*(nely+1)*(nelx+1)];
freedofs = setdiff(alldofs,fixeddofs);
%% PREPARE FILTER
iH = ones(nelx*nely*(2*(ceil(rmin)-1)+1)^2,1);
jH = ones(size(iH));
sH = zeros(size(iH));
k = 0;
for i1 = 1:nelx
  for j1 = 1:nely
    e1 = (i1-1)*nely+j1;
    for i2 = max(i1-(ceil(rmin)-1),1):min(i1+(ceil(rmin)-1),nelx)
      for j2 = max(j1-(ceil(rmin)-1),1):min(j1+(ceil(rmin)-1),nely)
        e2 = (i2-1)*nely+j2;
        k = k+1;
        iH(k) = e1;
        jH(k) = e2;
        sH(k) = max(0,rmin-sqrt((i1-i2)^2+(j1-j2)^2));
      end
    end
  end
end
H = sparse(iH,jH,sH);
Hs = sum(H,2);
%% INITIALIZE ITERATION
x = repmat(volfrac,nely,nelx);
xPhys = x;
loop = 0;
change = 1;
%% START ITERATION
while change > 0.01
  loop = loop + 1;
  %% FE-ANALYSIS
  sK = reshape(KE(:)*(Emin+xPhys(:)'.^penal*(E0-Emin)),64*nelx*nely,1);
  K = sparse(iK,jK,sK); 
  K = (K+K')/2;
  U(freedofs, :) = K(freedofs,freedofs)\F(freedofs, :);
  %% OBJECTIVE FUNCTION AND SENSITIVITY ANALYSIS  
  c=0;
  dc=0;
  for i = 1:size(F,2)
  Ui = U(:,i);
  ce = reshape(sum((Ui(edofMat)*KE).*Ui(edofMat),2), nely,nelx);
  c = c + sum(sum((Emin+xPhys.^penal*(E0-Emin)).*ce));
  dc = dc - penal*(E0-Emin)*xPhys.^(penal-1).*ce;
  end
  dv = ones(nely,nelx);
  %% FILTERING/MODIFICATION OF SENSITIVITIES
  if ft == 1
    dc(:) = H*(x(:).*dc(:))./Hs./max(1e-3,x(:));
  elseif ft == 2
    dc(:) = H*(dc(:)./Hs);
    dv(:) = H*(dv(:)./Hs);
  end
  %% OPTIMALITY CRITERIA UPDATE OF DESIGN VARIABLES AND PHYSICAL DENSITIES
  l1 = 0; l2 = 1e9; move = 0.2;
  while (l2-l1)/(l1+l2) > 1e-3
    lmid = 0.5*(l2+l1);
    xnew = max(0,max(x-move,min(1,min(x+move,x.*sqrt(-dc./dv/lmid)))));
    xPhys = xnew;
    if sum(xPhys(:)) > volfrac*nelx*nely 
        l1 = lmid; 
    else 
        l2 = lmid; 
    end
  end
  change = max(abs(xnew(:)-x(:)));
  x = xnew;
  %% PRINT RESULTS
  fprintf(' It.:%5i Obj.:%11.4f Vol.:%7.3f ch.:%7.3f\n',loop,c, mean(xPhys(:)),change);
  %% PLOT DENSITIES
  colormap(gray); imagesc(1-xPhys); caxis([0 1]); axis equal; axis off; drawnow;
  filename = sprintf('%s%s%s','image/testAnimated',num2str(loop),'.jpg');
  saveas(gcf, filename); 
end
%
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
% This Matlab code was written by E. Andreassen, A. Clausen, M. Schevenels,%
% B. S. Lazarov and O. Sigmund,  Department of Solid  Mechanics,           %
%  Technical University of Denmark,                                        %
%  DK-2800 Lyngby, Denmark.                                                %
% Please sent your comments to: sigmund@fam.dtu.dk                         %
%                                                                          %
% The code is intended for educational purposes and theoretical details    %
% are discussed in the paper                                               %
% "Efficient topology optimization in MATLAB using 88 lines of code,       %
% E. Andreassen, A. Clausen, M. Schevenels,                                %
% B. S. Lazarov and O. Sigmund, Struct Multidisc Optim, 2010               %
% This version is based on earlier 99-line code                            %
% by Ole Sigmund (2001), Structural and Multidisciplinary Optimization,    %
% Vol 21, pp. 120--127.                                                    %
%                                                                          %
% The code as well as a postscript version of the paper can be             %
% downloaded from the web-site: http://www.topopt.dtu.dk                   %
%                                                                          %
% Disclaimer:                                                              %
% The authors reserves all rights but do not guaranty that the code is     %
% free from errors. Furthermore, we shall not be liable in any event       %
% caused by the use of the program.                                        %
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

Created file 'D:\Robot\Python Program\MAE 598 Design Optimization\Homework\MAE-598-Design-Optimization\top88.m'.


Then we can use this command `top88(150,50,0.6,3,2,1)` to have the topology optimization, the calculation results are as below:

 - It.:    1 Obj.:   674.3033 Vol.:  0.600 ch.:  0.200
 - It.:    2 Obj.:   399.0123 Vol.:  0.600 ch.:  0.200
 - It.:    3 Obj.:   293.2249 Vol.:  0.600 ch.:  0.200
 - It.:    4 Obj.:   272.1214 Vol.:  0.600 ch.:  0.200
 - It.:    5 Obj.:   259.5088 Vol.:  0.600 ch.:  0.200
 - It.:    6 Obj.:   249.1546 Vol.:  0.600 ch.:  0.200
 - It.:    7 Obj.:   240.4096 Vol.:  0.600 ch.:  0.200
 - It.:    8 Obj.:   234.0594 Vol.:  0.600 ch.:  0.200
 - It.:    9 Obj.:   229.7007 Vol.:  0.600 ch.:  0.177
 - It.:   10 Obj.:   226.2924 Vol.:  0.600 ch.:  0.179
 - It.:   11 Obj.:   223.6211 Vol.:  0.600 ch.:  0.199
 - It.:   12 Obj.:   221.1959 Vol.:  0.600 ch.:  0.200
 - It.:   13 Obj.:   218.8707 Vol.:  0.600 ch.:  0.172
 - It.:   14 Obj.:   216.7615 Vol.:  0.600 ch.:  0.191
 - It.:   15 Obj.:   214.8182 Vol.:  0.600 ch.:  0.175
 - It.:   16 Obj.:   212.8766 Vol.:  0.600 ch.:  0.143
 - It.:   17 Obj.:   210.8994 Vol.:  0.600 ch.:  0.155
 - It.:   18 Obj.:   208.6959 Vol.:  0.600 ch.:  0.147
 - It.:   19 Obj.:   206.6410 Vol.:  0.600 ch.:  0.179
 - It.:   20 Obj.:   204.2141 Vol.:  0.600 ch.:  0.198
 - It.:   21 Obj.:   201.6323 Vol.:  0.600 ch.:  0.200
 - It.:   22 Obj.:   199.0959 Vol.:  0.600 ch.:  0.194
 - It.:   23 Obj.:   197.0404 Vol.:  0.600 ch.:  0.151
 - It.:   24 Obj.:   195.7017 Vol.:  0.600 ch.:  0.121
 - It.:   25 Obj.:   194.8060 Vol.:  0.600 ch.:  0.108
 - It.:   26 Obj.:   194.1414 Vol.:  0.600 ch.:  0.098
 - It.:   27 Obj.:   193.7482 Vol.:  0.600 ch.:  0.096
 - It.:   28 Obj.:   193.3959 Vol.:  0.600 ch.:  0.095
 - It.:   29 Obj.:   193.1822 Vol.:  0.600 ch.:  0.091
 - It.:   30 Obj.:   193.0257 Vol.:  0.600 ch.:  0.092
 - It.:   31 Obj.:   192.8681 Vol.:  0.600 ch.:  0.092
 - It.:   32 Obj.:   192.6800 Vol.:  0.600 ch.:  0.087
 - It.:   33 Obj.:   192.6333 Vol.:  0.600 ch.:  0.081
 - It.:   34 Obj.:   192.5349 Vol.:  0.600 ch.:  0.078
 - It.:   35 Obj.:   192.4568 Vol.:  0.600 ch.:  0.078
 - It.:   36 Obj.:   192.3804 Vol.:  0.600 ch.:  0.075
 - It.:   37 Obj.:   192.3182 Vol.:  0.600 ch.:  0.069
 - It.:   38 Obj.:   192.2567 Vol.:  0.600 ch.:  0.066
 - It.:   39 Obj.:   192.2039 Vol.:  0.600 ch.:  0.063
 - It.:   40 Obj.:   192.1565 Vol.:  0.600 ch.:  0.061
 - It.:   41 Obj.:   192.1157 Vol.:  0.600 ch.:  0.056
 - It.:   42 Obj.:   192.1468 Vol.:  0.600 ch.:  0.054
 - It.:   43 Obj.:   192.1297 Vol.:  0.600 ch.:  0.054
 - It.:   44 Obj.:   192.0423 Vol.:  0.600 ch.:  0.050
 - It.:   45 Obj.:   192.0709 Vol.:  0.600 ch.:  0.049
 - It.:   46 Obj.:   192.0489 Vol.:  0.600 ch.:  0.049
 - It.:   47 Obj.:   192.0345 Vol.:  0.600 ch.:  0.045
 - It.:   48 Obj.:   192.0110 Vol.:  0.600 ch.:  0.043
 - It.:   49 Obj.:   191.9939 Vol.:  0.600 ch.:  0.041
 - It.:   50 Obj.:   191.9714 Vol.:  0.600 ch.:  0.039
 - It.:   51 Obj.:   191.9508 Vol.:  0.600 ch.:  0.049
 - It.:   52 Obj.:   191.9312 Vol.:  0.600 ch.:  0.048
 - It.:   53 Obj.:   191.9152 Vol.:  0.600 ch.:  0.041
 - It.:   54 Obj.:   191.9737 Vol.:  0.600 ch.:  0.037
 - It.:   55 Obj.:   191.9054 Vol.:  0.600 ch.:  0.053
 - It.:   56 Obj.:   191.9626 Vol.:  0.600 ch.:  0.070
 - It.:   57 Obj.:   191.8816 Vol.:  0.600 ch.:  0.064
 - It.:   58 Obj.:   191.9414 Vol.:  0.600 ch.:  0.051
 - It.:   59 Obj.:   191.9342 Vol.:  0.600 ch.:  0.041
 - It.:   60 Obj.:   191.8669 Vol.:  0.600 ch.:  0.033
 - It.:   61 Obj.:   191.9130 Vol.:  0.600 ch.:  0.034
 - It.:   62 Obj.:   191.9143 Vol.:  0.600 ch.:  0.029
 - It.:   63 Obj.:   191.8443 Vol.:  0.600 ch.:  0.027
 - It.:   64 Obj.:   191.8998 Vol.:  0.600 ch.:  0.058
 - It.:   65 Obj.:   191.8998 Vol.:  0.600 ch.:  0.065
 - It.:   66 Obj.:   191.9084 Vol.:  0.600 ch.:  0.055
 - It.:   67 Obj.:   191.9040 Vol.:  0.600 ch.:  0.042
 - It.:   68 Obj.:   191.8282 Vol.:  0.600 ch.:  0.031
 - It.:   69 Obj.:   191.8786 Vol.:  0.600 ch.:  0.024
 - It.:   70 Obj.:   191.8774 Vol.:  0.600 ch.:  0.021
 - It.:   71 Obj.:   191.8820 Vol.:  0.600 ch.:  0.023
 - It.:   72 Obj.:   191.8786 Vol.:  0.600 ch.:  0.035
 - It.:   73 Obj.:   191.8794 Vol.:  0.600 ch.:  0.031
 - It.:   74 Obj.:   191.8762 Vol.:  0.600 ch.:  0.021
 - It.:   75 Obj.:   191.8742 Vol.:  0.600 ch.:  0.017
 - It.:   76 Obj.:   191.8695 Vol.:  0.600 ch.:  0.017
 - It.:   77 Obj.:   191.8664 Vol.:  0.600 ch.:  0.046
 - It.:   78 Obj.:   191.8619 Vol.:  0.600 ch.:  0.090
 - It.:   79 Obj.:   191.8596 Vol.:  0.600 ch.:  0.096
 - It.:   80 Obj.:   191.8551 Vol.:  0.600 ch.:  0.076
 - It.:   81 Obj.:   191.8506 Vol.:  0.600 ch.:  0.053
 - It.:   82 Obj.:   191.8463 Vol.:  0.600 ch.:  0.037
 - It.:   83 Obj.:   191.8423 Vol.:  0.600 ch.:  0.026
 - It.:   84 Obj.:   191.8383 Vol.:  0.600 ch.:  0.033
 - It.:   85 Obj.:   191.8356 Vol.:  0.600 ch.:  0.069
 - It.:   86 Obj.:   191.8327 Vol.:  0.600 ch.:  0.075
 - It.:   87 Obj.:   191.8294 Vol.:  0.600 ch.:  0.060
 - It.:   88 Obj.:   191.8266 Vol.:  0.600 ch.:  0.041
 - It.:   89 Obj.:   191.8243 Vol.:  0.600 ch.:  0.028
 - It.:   90 Obj.:   191.8224 Vol.:  0.600 ch.:  0.019
 - It.:   91 Obj.:   191.8211 Vol.:  0.600 ch.:  0.014
 - It.:   92 Obj.:   191.8199 Vol.:  0.600 ch.:  0.012
 - It.:   93 Obj.:   191.8188 Vol.:  0.600 ch.:  0.013
 - It.:   94 Obj.:   191.8941 Vol.:  0.600 ch.:  0.017
 - It.:   95 Obj.:   191.8378 Vol.:  0.600 ch.:  0.057
 - It.:   96 Obj.:   191.8358 Vol.:  0.600 ch.:  0.080
 - It.:   97 Obj.:   191.8244 Vol.:  0.600 ch.:  0.071
 - It.:   98 Obj.:   191.8986 Vol.:  0.600 ch.:  0.051
 - It.:   99 Obj.:   191.8392 Vol.:  0.600 ch.:  0.034
 - It.:  100 Obj.:   191.8371 Vol.:  0.600 ch.:  0.023
 - It.:  101 Obj.:   191.8239 Vol.:  0.600 ch.:  0.015
 - It.:  102 Obj.:   191.8997 Vol.:  0.600 ch.:  0.012
 - It.:  103 Obj.:   191.8406 Vol.:  0.600 ch.:  0.011
 - It.:  104 Obj.:   191.8407 Vol.:  0.600 ch.:  0.017
 - It.:  105 Obj.:   191.8297 Vol.:  0.600 ch.:  0.016
 - It.:  106 Obj.:   191.8314 Vol.:  0.600 ch.:  0.013
 - It.:  107 Obj.:   191.9041 Vol.:  0.600 ch.:  0.011
 - It.:  108 Obj.:   191.8538 Vol.:  0.600 ch.:  0.011
 - It.:  109 Obj.:   191.8522 Vol.:  0.600 ch.:  0.011
 - It.:  110 Obj.:   191.8455 Vol.:  0.600 ch.:  0.011
 - It.:  111 Obj.:   191.8459 Vol.:  0.600 ch.:  0.011
 - It.:  112 Obj.:   191.8452 Vol.:  0.600 ch.:  0.012
 - It.:  113 Obj.:   191.8479 Vol.:  0.600 ch.:  0.011
 - It.:  114 Obj.:   191.8515 Vol.:  0.600 ch.:  0.018
 - It.:  115 Obj.:   191.8570 Vol.:  0.600 ch.:  0.022
 - It.:  116 Obj.:   191.8647 Vol.:  0.600 ch.:  0.017
 - It.:  117 Obj.:   191.8746 Vol.:  0.600 ch.:  0.012
 - It.:  118 Obj.:   191.8867 Vol.:  0.600 ch.:  0.013
 - It.:  119 Obj.:   191.9012 Vol.:  0.600 ch.:  0.015
 - It.:  120 Obj.:   191.9170 Vol.:  0.600 ch.:  0.016
 - It.:  121 Obj.:   191.9361 Vol.:  0.600 ch.:  0.018
 - It.:  122 Obj.:   191.9583 Vol.:  0.600 ch.:  0.020
 - It.:  123 Obj.:   191.9864 Vol.:  0.600 ch.:  0.021
 - It.:  124 Obj.:   191.9423 Vol.:  0.600 ch.:  0.023
 - It.:  125 Obj.:   191.9565 Vol.:  0.600 ch.:  0.027
 - It.:  126 Obj.:   191.9791 Vol.:  0.600 ch.:  0.032
 - It.:  127 Obj.:   192.0216 Vol.:  0.600 ch.:  0.037
 - It.:  128 Obj.:   191.9986 Vol.:  0.600 ch.:  0.049
 - It.:  129 Obj.:   192.0447 Vol.:  0.600 ch.:  0.064
 - It.:  130 Obj.:   192.0342 Vol.:  0.600 ch.:  0.086
 - It.:  131 Obj.:   192.0239 Vol.:  0.600 ch.:  0.116
 - It.:  132 Obj.:   191.9118 Vol.:  0.600 ch.:  0.141
 - It.:  133 Obj.:   191.7172 Vol.:  0.600 ch.:  0.129
 - It.:  134 Obj.:   191.4815 Vol.:  0.600 ch.:  0.141
 - It.:  135 Obj.:   191.2754 Vol.:  0.600 ch.:  0.116
 - It.:  136 Obj.:   191.1652 Vol.:  0.600 ch.:  0.043
 - It.:  137 Obj.:   191.1301 Vol.:  0.600 ch.:  0.031
 - It.:  138 Obj.:   191.1499 Vol.:  0.600 ch.:  0.045
 - It.:  139 Obj.:   191.1346 Vol.:  0.600 ch.:  0.053
 - It.:  140 Obj.:   191.1910 Vol.:  0.600 ch.:  0.044
 - It.:  141 Obj.:   191.1271 Vol.:  0.600 ch.:  0.031
 - It.:  142 Obj.:   191.1800 Vol.:  0.600 ch.:  0.022
 - It.:  143 Obj.:   191.1770 Vol.:  0.600 ch.:  0.023
 - It.:  144 Obj.:   191.1144 Vol.:  0.600 ch.:  0.030
 - It.:  145 Obj.:   191.1552 Vol.:  0.600 ch.:  0.026
 - It.:  146 Obj.:   191.1552 Vol.:  0.600 ch.:  0.020
 - It.:  147 Obj.:   191.1590 Vol.:  0.600 ch.:  0.019
 - It.:  148 Obj.:   191.1558 Vol.:  0.600 ch.:  0.018
 - It.:  149 Obj.:   191.1531 Vol.:  0.600 ch.:  0.034
 - It.:  150 Obj.:   191.1477 Vol.:  0.600 ch.:  0.105
 - It.:  151 Obj.:   191.1426 Vol.:  0.600 ch.:  0.138
 - It.:  152 Obj.:   191.1353 Vol.:  0.600 ch.:  0.117
 - It.:  153 Obj.:   191.1279 Vol.:  0.600 ch.:  0.082
 - It.:  154 Obj.:   191.1205 Vol.:  0.600 ch.:  0.053
 - It.:  155 Obj.:   191.1151 Vol.:  0.600 ch.:  0.033
 - It.:  156 Obj.:   191.1106 Vol.:  0.600 ch.:  0.021
 - It.:  157 Obj.:   191.1071 Vol.:  0.600 ch.:  0.014
 - It.:  158 Obj.:   191.1760 Vol.:  0.600 ch.:  0.012
 - It.:  159 Obj.:   191.1251 Vol.:  0.600 ch.:  0.027
 - It.:  160 Obj.:   191.1219 Vol.:  0.600 ch.:  0.030
 - It.:  161 Obj.:   191.1072 Vol.:  0.600 ch.:  0.023
 - It.:  162 Obj.:   191.1033 Vol.:  0.600 ch.:  0.014
 - It.:  163 Obj.:   191.1675 Vol.:  0.600 ch.:  0.019
 - It.:  164 Obj.:   191.1165 Vol.:  0.600 ch.:  0.019
 - It.:  165 Obj.:   191.1109 Vol.:  0.600 ch.:  0.034
 - It.:  166 Obj.:   191.1695 Vol.:  0.600 ch.:  0.036
 - It.:  167 Obj.:   191.1161 Vol.:  0.600 ch.:  0.027
 - It.:  168 Obj.:   191.1096 Vol.:  0.600 ch.:  0.017
 - It.:  169 Obj.:   191.1676 Vol.:  0.600 ch.:  0.011
 - It.:  170 Obj.:   191.1137 Vol.:  0.600 ch.:  0.008

Final geometry and animation for the topology optimization progress is shown as below. (If animation can not show on this page, please click this link: https://github.com/dayanbatuofu/MAE-598-Design-Optimization/blob/main/topology%20optimization.gif)

![Image](https://github.com/dayanbatuofu/MAE-598-Design-Optimization/raw/main/final%20gemotry.jpg)

![Image](https://github.com/dayanbatuofu/MAE-598-Design-Optimization/raw/main/topology%20optimization.gif)